# Imports

In [14]:
import sys
import os
from google.colab import drive
from google.colab import files
from dotenv import load_dotenv
import json
import warnings
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
%matplotlib inline

# Bootstrap

In [15]:
np.random.seed(31071967)

# Find and load the .env file from the current or parent directories
load_dotenv()

drive.mount('/content/drive')

with open(f"{os.getenv('PROJECT_PATH')}/src/config.json", 'r') as f:
    project_config = json.load(f)
    project_config.pop('_comment', None)
    project_config.pop('_note', None)
    f.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Download TKL data from YF

In [16]:
tickers_yf = {
    "y"     : f"{project_config['TKL']}",
 }

desired_order = [
    "Date",
    "y",
  ]

In [17]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr

from datetime import date, timedelta
end_date = date.today() - timedelta(days=1)
start_date = end_date - timedelta(days=int(project_config["HISTORY_DEPTH"]))

if project_config['TKL'] == 'TNYA':
  start_date = pd.to_datetime("30.07.2022", format="%d.%m.%Y")

# ---- DOWNLOAD FROM YAHOO FINANCE ----
ts_yf = yf.download(
    tickers=list(tickers_yf.values()),
    start=start_date,
    end=end_date,
    auto_adjust=True
)["Close"]

# rename columns to readable names
rename_map = {v: k for k, v in tickers_yf.items()}
ts_yf = ts_yf.rename(columns=rename_map)

# Fill missing daily values for macro data (monthly)
ts_yf = ts_yf.fillna(method='ffill').fillna(method='bfill')
ts_yf = ts_yf.reset_index().rename(columns={"Date": "Date",})

print(f"\n\nDataset for y={project_config['TKL']}")
display(ts_yf.head(1))
display(ts_yf.tail(1))
ts_yf.info()
df = ts_yf.copy()

[*********************100%***********************]  1 of 1 completed



Dataset for y=NVDA


Ticker,Date,y
0,2006-10-23,0.48256


Ticker,Date,y
4820,2025-12-19,180.990005


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4821 entries, 0 to 4820
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    4821 non-null   datetime64[ns]
 1   y       4821 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 75.5 KB


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# -----------------------
# Parameters
# -----------------------
STEP_DAYS = 10
WINDOW_DAYS = 260
FUTURE_DAYS = 66

BUY_TH = 0.10     # +10%
SELL_TH = -0.10   # -10%

OUT_DIR = f"{os.getenv('PROJECT_PATH')}{project_config['images_directory']}{project_config['TKL']}_graphs"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------
# Prepare data
# -----------------------
df = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

results = []

# -----------------------
# Main loop
# -----------------------
for start_idx in range(0, len(df) - WINDOW_DAYS - FUTURE_DAYS, STEP_DAYS):

    graph_df = df.iloc[start_idx:start_idx + WINDOW_DAYS]
    future_df = df.iloc[start_idx + WINDOW_DAYS:
                         start_idx + WINDOW_DAYS + FUTURE_DAYS]

    start_date = graph_df['Date'].iloc[0].date()
    end_date = graph_df['Date'].iloc[-1].date()

    # -----------------------
    # Recommendation logic
    # -----------------------
    last_price = graph_df['y'].iloc[-1]
    future_price = future_df['y'].max()

    ret = (future_price - last_price) / last_price

    if ret > BUY_TH:
        rec = "BUY"
    elif ret < SELL_TH:
        rec = "SALE"
    else:
        rec = "KEEP"

    # -----------------------
    # Plot (PURE SHAPE)
    # -----------------------
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(graph_df['y'])

    # remove all axis visuals
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.grid(False)

    # optional: remove frame completely
    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.tight_layout()

    fname = f"{project_config['TKL']}_{start_date}_{end_date}_{rec}.png"
    plt.savefig(os.path.join(OUT_DIR, fname), bbox_inches="tight", pad_inches=0)
    plt.close()

    # -----------------------
    # Store metadata
    # -----------------------
    results.append({
        "start_date": start_date,
        "end_date": end_date,
        "last_price": last_price,
        "future_price": future_price,
        "66d_return": ret,
        "recommendation": rec,
        "file": fname
    })

# -----------------------
# Results DataFrame
# -----------------------
df_results = pd.DataFrame(results)

In [20]:
df_results

,start_date,end_date,last_price,future_price,66d_return,recommendation,file
0,2006-10-23,2007-11-02,0.842112,0.868701,0.031573,KEEP,NVDA_2006-10-23_2007-11-02_KEEP.png
1,2006-11-06,2007-11-16,0.743782,0.831110,0.117412,BUY,NVDA_2006-11-06_2007-11-16_BUY.png
2,2006-11-20,2007-12-03,0.733468,0.831110,0.133125,BUY,NVDA_2006-11-20_2007-12-03_BUY.png
3,2006-12-05,2007-12-17,0.762806,0.831110,0.089543,KEEP,NVDA_2006-12-05_2007-12-17_KEEP.png
4,2006-12-19,2008-01-02,0.756618,0.750658,-0.007877,KEEP,NVDA_2006-12-19_2008-01-02_KEEP.png
...,...,...,...,...,...,...,...
445,2024-07-01,2025-07-15,170.680862,192.559280,0.128183,BUY,NVDA_2024-07-01_2025-07-15_BUY.png
446,2024-07-16,2025-07-29,175.490326,207.028473,0.179714,BUY,NVDA_2024-07-16_2025-07-29_BUY.png
447,2024-07-30,2025-08-12,183.139481,207.028473,0.130442,BUY,NVDA_2024-07-30_2025-08-12_BUY.png
448,2024-08-13,2025-08-26,181.749634,207.028473,0.139086,BUY,NVDA_2024-08-13_2025-08-26_BUY.png


In [ ]:
# import pandas as pd

# # Define the evaluation period (66 days) and threshold (5%)
# horizon = 66
# threshold = 0.05
# step = 22
# window_size = 260

# classifications = []

# # We stop 'horizon' days before the end of the dataframe to ensure we have future data to check
# for i in range(0, len(df) - window_size - horizon, step):
#     # Current window end price
#     p_end = df.iloc[i + window_size - 1]['y']

#     # Price 66 days into the future
#     p_future = df.iloc[i + window_size + horizon - 1]['y']

#     # Calculate return
#     pct_change = (p_future - p_end) / p_end

#     # Classify
#     if pct_change > threshold:
#         recommendation = "Buy"
#     elif pct_change < -threshold:
#         recommendation = "Sale"
#     else:
#         recommendation = "Keep"

#     # Store result mapping to the filename
#     start_date = df.index[i].strftime('%Y-%m-%d')
#     classifications.append({
#         'Graph_File': f'nvda_window_{start_date}.png',
#         'Recommendation_nextQ': recommendation
#     })

# # Save results to a CSV
# class_df = pd.DataFrame(classifications)
# class_df.to_csv('graph_classifications.csv', index=False)

# print(class_df.head(10))